In [0]:
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd

py.offline.init_notebook_mode(connected=True)

#import data

#datapath = "C:\Users\Dan Steffy\Dropbox\Research\VRPVIS\\"
datapath = "/Users/TomCreo/Desktop/"

df_ArrivalTimes = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalTimes")
df_Locations = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Locations")
df_ArrivalLoad = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalLoad")
df_Demand = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Demand")
df_Requests = pd.read_csv(str(datapath+"Google_Campus_Data_217_Shifted.csv"))

#to store requestID associated with each stop
VehRequestID = []

#to store latitude associated with each stop
VehLat = []

#to store longitudes associated with each stop
VehLong = []

#marker fill based on pickup or dropoff
VehFill = []

#hovertext RequestID
VehText = []


#create lists of lists
for v in range(30):
    VehRequestID.append([])
    VehLat.append([])
    VehLong.append([])
    VehFill.append([])
    VehText.append([])
    
#calculate request ID associated with each stop
#extract latitude/longitude from requestID
#assign marker fill: pickup = 'circle', dropoff = 'circle-open'

clist = df_Locations.columns
for v in range(30):
    for i, location in enumerate(df_Locations[clist[v]].dropna(),start=1):
        vehdemand =  df_Demand[clist[v]].dropna()[i]
        if vehdemand == 1:
            VehRequestID[v].append(location/2)
            VehLat[v].append(df_Requests.fromLatitude[location/2])
            VehLong[v].append(df_Requests.fromLongitude[location/2])
            VehFill[v].append('circle')
            VehText[v].append('Pickup RequestID '+str(int(location/2)))
        elif vehdemand == -1:
            VehRequestID[v].append((location-1)/2)
            VehLat[v].append(df_Requests.toLatitude[(location-1)/2])
            VehLong[v].append(df_Requests.toLongitude[(location-1)/2])
            VehFill[v].append('circle-open')
            VehText[v].append('Dropoff RequestID '+str(int((location-1)/2)))

            
#define minute by minute interval between first arrival time and last in solution data
interval = range(27010,37388,60)

#create list of positions associated with interval for vehicle
VehPosition = []
            
#minute by minute position for Veh0 all stops
for i in range(len(VehLong[0])-1):
    for z in interval:
        if df_ArrivalTimes[clist[0]].dropna()[i+1] <= z <= df_ArrivalTimes[clist[0]].dropna()[i+2]:
            t = (z - df_ArrivalTimes[clist[0]].dropna()[i+1]) / (df_ArrivalTimes[clist[0]].dropna()[i+2]-df_ArrivalTimes[clist[0]].dropna()[i+1])
            x = VehLong[0][i] + (VehLong[0][i+1]-VehLong[0][i])*t
            y = VehLat[0][i] + (VehLat[0][i+1]-VehLat[0][i])*t
            VehPosition.append((x, y, z))
            
#extract x, y, z coords from positions
positionx = [i[0] for i in VehPosition]
positiony = [i[1] for i in VehPosition]
positionz = [i[2] for i in VehPosition]
    

data = []
for v in range(30):
    trace = go.Scatter3d(
        x=VehLong[v],
        y=VehLat[v],
        z=df_ArrivalTimes[clist[v]].dropna(),
        name='Vehicle' + str(v),
        mode='lines+markers',
        marker=dict(
            size=6,
            symbol=VehFill[v],
            line=dict(
                width=0.5
            ),
            opacity=0.7
        ),
        line=dict(),
        text=VehText[v]
    )
    data.append(trace)

    
#trace to check positions against veh0 route
traceposition = go.Scatter3d(
        x=positionx,
        y=positiony,
        z=positionz,
        name='Veh0 positions',
        mode='markers',
        marker=dict(
            size=3,
            symbol='square',
            line=dict(
                width=0.5
            ),
            opacity=0.7
        ),
        line=dict(),
    )
data.append(traceposition)
    
    
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    legend=dict(
        x=0,
        y=0,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='#000'
        ),
        bgcolor='#E2E2E2',
        bordercolor='#FFFFFF',
        borderwidth=2
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)



In [15]:
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import itertools
from haversine import haversine
from scipy import stats


py.offline.init_notebook_mode(connected=True)

#import data

#datapath = "C:\Users\Dan Steffy\Dropbox\Research\VRPVIS\\"
datapath = "/Users/TomCreo/Desktop/"

df_ArrivalTimes = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalTimes")
df_Locations = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Locations")
df_ArrivalLoad = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalLoad")
df_Demand = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Demand")
df_Requests = pd.read_csv(str(datapath+"Google_Campus_Data_217_Shifted.csv"))

#to store requestID associated with each stop
VehRequestID = []

#to store latitude associated with each stop
VehLat = []

#to store longitudes associated with each stop
VehLong = []

#marker fill based on pickup or dropoff
VehFill = []

#hovertext RequestID
VehText = []


#create lists of lists
for v in range(30):
    VehRequestID.append([])
    VehLat.append([])
    VehLong.append([])
    VehFill.append([])
    VehText.append([])
    
    
#calculate request ID associated with each stop
#extract latitude/longitude from requestID
#assign marker fill: pickup = 'circle', dropoff = 'circle-open'

clist = df_Locations.columns
for v in range(30):
    for i, location in enumerate(df_Locations[clist[v]].dropna(),start=1):
        vehdemand =  df_Demand[clist[v]].dropna()[i]
        if vehdemand == 1:
            VehRequestID[v].append(location/2)
            VehLat[v].append(df_Requests.fromLatitude[location/2])
            VehLong[v].append(df_Requests.fromLongitude[location/2])
            VehFill[v].append('circle')
            VehText[v].append('Pickup RequestID '+str(int(location/2)))
        elif vehdemand == -1:
            VehRequestID[v].append((location-1)/2)
            VehLat[v].append(df_Requests.toLatitude[(location-1)/2])
            VehLong[v].append(df_Requests.toLongitude[(location-1)/2])
            VehFill[v].append('circle-open')
            VehText[v].append('Dropoff RequestID '+str(int((location-1)/2)))

            
            
#define minute by minute interval between first arrival time and last in solution data
interval = range(27010,37388,60)

            
#minute by minute position for given vehicle between all stops
def position(VehNum):
    VehPosition = []
    for i in range(len(VehLong[VehNum])-1):
        for z in interval:
            if df_ArrivalTimes[clist[VehNum]].dropna()[i+1] <= z <= df_ArrivalTimes[clist[VehNum]].dropna()[i+2]:
                t = ((z - df_ArrivalTimes[clist[VehNum]].dropna()[i+1]) / 
                        (df_ArrivalTimes[clist[VehNum]].dropna()[i+2]-df_ArrivalTimes[clist[VehNum]].dropna()[i+1])
                    )
                x = VehLong[VehNum][i] + (VehLong[VehNum][i+1]-VehLong[VehNum][i])*t
                y = VehLat[VehNum][i] + (VehLat[VehNum][i+1]-VehLat[VehNum][i])*t
                VehPosition.append((x, y, z))
    return VehPosition;




#calculates avg location for 2 specified vehicles and measures distance between them
#returns distance between avg locations in km

def avg_loc_dist(VehNum1,VehNum2):
    AvgLong1 = sum(VehLong[VehNum1]) / len(VehLong[VehNum1])
    AvgLat1 = sum(VehLat[VehNum1]) / len(VehLat[VehNum1])
    AvgLoc1 = (AvgLong1, AvgLat1)

    AvgLong2 = sum(VehLong[VehNum2]) / len(VehLong[VehNum2])
    AvgLat2 = sum(VehLat[VehNum2]) / len(VehLat[VehNum2])
    AvgLoc2 = (AvgLong2, AvgLat2)
    
    dist = haversine(AvgLoc1, AvgLoc2)
    return dist;



#calculate distance in km for all combinations of 2 vehicles and find minimum overall vehicles
AvgLocDist = []
AvgLocMinDist = None

for pair in itertools.combinations(range(30), 2):
    CurrentDist = avg_loc_dist(*pair)
    AvgLocDist.append([pair, CurrentDist])
    if AvgLocMinDist == None:
        AvgLocMinDist = CurrentDist
    else:
        AvgLocMinDist = min(AvgLocMinDist,CurrentDist)


        
    
    
#calculates distance between 2 specified vehicles over time interval where both vehicles are active
#returns list of distance at each t; returns none if vehicles not active at same time
def min_dist(VehNum1,VehNum2):
    dist2 = []
    VehNum1Pos = position(VehNum1)
    VehNum2Pos = position(VehNum2)
    for t in interval:
        VehNum1Loc_t = [item for item in VehNum1Pos if item[2] == t]
        VehNum2Loc_t = [item for item in VehNum2Pos if item[2] == t]
        if (len(VehNum1Loc_t) != 0 and
            len(VehNum2Loc_t) != 0):
            VehNum1Loc = (VehNum1Loc_t[0][0],VehNum1Loc_t[0][1])
            VehNum2Loc = (VehNum2Loc_t[0][0],VehNum2Loc_t[0][1])
            dist2.append(((haversine(VehNum1Loc, VehNum2Loc))))
    if len(dist2) != 0:
        return dist2;
    else:
        return None;



#caclulates min distance for all pairs of vehicles
MinLocDist = []

#for pair in itertools.combinations(range(30), 2):
#    min_distance = min_dist(*pair)
#    MinLocDist.append([pair, min_distance])
    
#print MinLocDist

    
    
    
    
    
#calculates avg distance between 2 specified vehicles over time when both are active; returns distance
def avg_dist(VehNum1,VehNum2):
    Distance_t = min_dist(VehNum1,VehNum2)
    if Distance_t != None:
        DistSum = sum(Distance_t)
        AvgDist = DistSum / len(Distance_t)
        return AvgDist;
    else:
        return None;



#caclulates avg distance for all pairs of vehicles
AvgDist = []
           
for pair in itertools.combinations(range(30), 2):
    avg_distance = avg_dist(*pair)
    print([pair, avg_distance])
    AvgDist.append([pair, avg_distance])




#read in experiment output and calculate improvements
df_ExpData = pd.read_excel(str(datapath+"Data 435.xlsx"))

ReoptImprovement = []

for i in range(len(df_ExpData.FirstVehicle)-1):
    ReoptImprovement.append([(int(df_ExpData.FirstVehicle[i+1]), int(df_ExpData.SecondVehicle[i+1])), (int(df_ExpData.OriginalCost_s[i+1]) - int(df_ExpData.MIPTrueCost_s[i+1]))])
    
    



#create 2d scatterplots


ReoptImpY = []
VehPairs = []


for i, data in enumerate(ReoptImprovement):
    if AvgDist[i][1] != None:
        ReoptImpY.append(data[1])
        VehPairs.append(data[0])


#versus AvgDist
AvgDistX = [i[1] for i in AvgDist if i[1] != None]



slope1, intercept1, r_value, p_value, std_err = stats.linregress(AvgDistX,ReoptImpY)

line1 = slope1*np.array(AvgDistX)+intercept1



trace0 = go.Scatter(
    x = AvgDistX,
    y = ReoptImpY,
    mode = 'markers',
    text = VehPairs,
    name = 'Vehicle Route Pairs'
    )

trace1 = go.Scatter(
    x=AvgDistX,
    y=line1,
    mode='lines',
    name='Linear Fit'
    )

data1 = [trace0,trace1]

layout1 = go.Layout(
    title = 'Average Distance Between Route Pairs vs. Reoptimized Improvement',
    font=dict(family='Arial, sans-serif', size=12),
    xaxis = dict(
        title = 'Average Distance Between Route Pairs over Total Time Active (km)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=12
        ),
        tickfont=dict(
            family='Arial, sans-serif',
            size=10
        ),
    ),
    yaxis = dict(
        title = 'Route Cost Improvement upon Reoptimizing Route Pair (s)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=12
        ),
        tickfont=dict(
            family='Arial, sans-serif',
            size=10
        )
    )
)

fig1 = dict(data=data1, layout=layout1)

py.offline.iplot(fig1)

print "R value: "
print r_value


[(0, 1), 3.4179267472897967]
[(0, 2), 4.335346217308027]
[(0, 3), 6.344353915419859]
[(0, 4), 9.259352560907363]
[(0, 5), 7.1670721135988895]
[(0, 6), 5.283418082493766]
[(0, 7), 4.4972282928687735]
[(0, 8), 4.646546767464128]
[(0, 9), 4.0202590987491655]
[(0, 10), 7.247873006399171]
[(0, 11), 6.565125599154157]
[(0, 12), 6.231909523292151]
[(0, 13), 4.110061539014628]
[(0, 14), 5.116498811416313]
[(0, 15), 3.3245172569598593]
[(0, 16), 7.683036206869415]
[(0, 17), 3.811188679482232]
[(0, 18), 11.521832849150508]
[(0, 19), 2.34788989734957]
[(0, 20), 9.874894714196362]


/Users/TomCreo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:82: RuntimeWarning:

invalid value encountered in double_scalars



[(0, 21), 7.502145801899491]
[(0, 22), 1.895505537048907]
[(0, 23), 6.136864107437143]
[(0, 24), 5.350039496180091]
[(0, 25), 4.854157346478426]
[(0, 26), 6.725368163788052]
[(0, 27), 4.12435156701155]
[(0, 28), 3.595989783805174]
[(0, 29), 2.183665711877918]
[(1, 2), 4.62328437553025]
[(1, 3), 6.32263920074861]
[(1, 4), 8.039667663710848]
[(1, 5), 6.133275034011121]
[(1, 6), 5.459267595836833]
[(1, 7), 4.9434448252924845]
[(1, 8), 4.901830327900125]
[(1, 9), 5.718574575150115]
[(1, 10), 4.9543845715434305]
[(1, 11), 8.882332049244102]
[(1, 12), 7.533902145435223]
[(1, 13), 5.069733485770299]
[(1, 14), 5.147157453205401]
[(1, 15), 6.457002796081424]
[(1, 16), 9.359909966179739]
[(1, 17), 4.488496029661666]
[(1, 18), 10.859028527878822]
[(1, 19), 1.8645038760473254]
[(1, 20), 8.817617870622529]
[(1, 21), 3.6732337825934116]
[(1, 22), 3.048593519756316]
[(1, 23), 4.388395169224975]
[(1, 24), 4.85262116144586]
[(1, 25), 5.244507493179098]
[(1, 26), 6.163583042682658]
[(1, 27), 3.913198111

[(13, 15), 3.002366658488261]
[(13, 16), 4.929708843705698]
[(13, 17), 4.362574624535918]
[(13, 18), 8.45321943506024]
[(13, 19), 0.6565354171678458]
[(13, 20), 6.5050662100225844]
[(13, 21), 7.254149139414591]
[(13, 22), 3.0871648982595365]
[(13, 23), 5.296336358192851]
[(13, 24), 2.7048337582105977]
[(13, 25), 2.4984350626703304]
[(13, 26), 3.432251220642593]
[(13, 27), 1.8607582259837971]
[(13, 28), 3.4945112554701234]
[(13, 29), 2.403207897664988]
[(14, 15), 5.169115989162351]
[(14, 16), 5.5230362656784955]
[(14, 17), 5.346297363916125]
[(14, 18), 7.003218202280231]
[(14, 19), 1.609958412199666]
[(14, 20), 4.6258130219591544]
[(14, 21), 6.012233972526495]
[(14, 22), 6.68571176250995]
[(14, 23), 3.1889964167772553]
[(14, 24), 3.526799337407809]
[(14, 25), 2.932880941405204]
[(14, 26), 2.278443149144529]
[(14, 27), 4.313827068981149]
[(14, 28), 3.636293423795453]
[(14, 29), 5.973436954729364]
[(15, 16), 4.464280497012448]
[(15, 17), 4.596104478715225]
[(15, 18), 9.322131111279305]
[(

R value: 
0.006160208672165289
